<a href="https://colab.research.google.com/github/sagnikbiswas/EastKolkataWetlandsLandCover/blob/main/EKW_RandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

Installation

In [ ]:
%%capture
!pip install rasterio
!pip install geopandas
!pip install earthpy

Imports

In [ ]:
# For satellite image processing
import rasterio as rio
from rasterio.plot import show
from rasterio.enums import Resampling

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import geopandas as gpd

#For machine learning
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score,\
 matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# For data analysis
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors
import pandas as pd
import seaborn as sns


# Others
import os
import time


#Data

In [ ]:
esri_landcover_path = '/content/drive/MyDrive/Kolkata_image_data/ESRI Land Cover Classified/45Q_20200101-20210101.tif'
ls_8_2021_path = '/content/drive/MyDrive/Kolkata_image_data/2021-04-25/stacked_raster.tif'
wetland_bigger = '/content/drive/MyDrive/Kolkata_image_data/geojson_files/wetland_bigger.geojson'

#Satellite Image Processing Functions

In [ ]:
def clip_and_stack(image_paths, geojson, geom=None, clip_only=False):
  """
  image: list of paths to the images
  geojson: path to the geojson-like file which contains the geometry of the clip
  geom: geometry object
  clip_only: Don't Stack
  returns stacked clipped raster
  """
  if type(image_paths) != list:
    image_paths = [image_paths]
  
  with rio.open(image_paths[0]) as f:
    crs = f.meta['crs']

  if type(geom) == type(None):
    geom = gpd.read_file(geojson)
    geom = geom.to_crs(crs)
    geom = geom.geometry

  clipped_path_list = es.crop_all(image_paths, '.', geom, overwrite=True)

  if clip_only:
    return clipped_path_list

  stacked, meta = es.stack(clipped_path_list, nodata=-9999)

  return stacked

In [ ]:
def resample(original, ht=0, wd=0, method=Resampling.bilinear, from_file=None):
  if from_file:
    with rio.open(from_file) as f:
      wd = f.meta['width']
      ht = f.meta['height']

  with rio.open(original) as dataset:
    # resample data to target shape
    resampled_data = dataset.read(
        out_shape=(
            dataset.count,
            int(ht),
            int(wd)
        ),
        resampling=method
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / resampled_data.shape[-1]),
        (dataset.height / resampled_data.shape[-2])
    )
    return resampled_data, transform

In [ ]:
def get_arr_from_file(path):
  with rio.open(path) as f:
    return f.read()

In [ ]:
def trim_edges(arr, n):
  return arr[:, n:-n, n:-n]

In [ ]:
# Appends pairwise normalized difference values to a raster array. input shape must be (pixel_num, band_num)
def add_nd(np_arr):
  arr64 = np_arr.astype('float64')
  n = len(np_arr[0])
  for b1 in range(n):
    for b2 in range(b1+1, n):
      nd = es.normalized_diff(arr64[:,b1], arr64[:,b2])
      arr64 = np.concatenate((arr64, nd.reshape(nd.shape[0], 1)), axis = 1)
  arr64 = np.nan_to_num(arr64)
  return arr64

In [ ]:
cmap = colors.ListedColormap(['#ffffff', '#429AE1', '#397E48', '#88AF52', '#7987C6', '#E49634', '#DFC25A', '#C4281B', '#A59B8F', '#F3F6FB', '#ffffff'])
boundaries = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5]
norm = colors.BoundaryNorm(boundaries, cmap.N, clip=True)

#Workspace

In [ ]:
class Workspace:
  # 
  def __init__(self, X_path, y_path, clip_area=None, resample_needed=False, trim_needed=False, delete_bands=False):
    self.X_path = X_path
    self.y_path = y_path
    if clip_area:
      self.X_path = clip_and_stack([self.X_path], geojson=clip_area, clip_only=True)[0]
      self.y_path = clip_and_stack([self.y_path], geojson=clip_area, clip_only=True)[0]
    
    self.X = get_arr_from_file(self.X_path)
    if delete_bands:
      self.X = np.delete(self.X, delete_bands, 0)
    # This resamples y according to X
    if resample_needed:
      self.y, tr = resample(self.y_path, method=Resampling.mode, from_file=self.X_path)
    else:
      self.y = get_arr_from_file(self.y_path)

    if trim_needed:
      self.X = trim_edges(self.X, trim_needed)
      self.y = trim_edges(self.y, trim_needed)

    assert(self.X.shape[-1] == self.y.shape[-1] and self.X.shape[-2] == self.y.shape[-2])

    self.original_shape = self.X.shape

  # Flatten, categorize and augment with nd bands
  def process(self, flatten=True, ohe=True, add_nd_needed=True):
    if flatten:
      self.X = np.transpose(self.X.reshape(self.X.shape[0], self.X.shape[-1]*self.X.shape[-2]))
    
    if ohe:
      self.y = pd.get_dummies(self.y.flatten())
      self.categories = self.y.columns

    if add_nd_needed:
      self.X = add_nd(self.X)

  def split_datasets(self, test_size=0.3):
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y,
                                                  test_size=test_size, random_state=42)
    return self.X_train, self.X_test, self.y_train, self.y_test

  def get_fitted_model(self, n_estimators=100, max_leaf_nodes=None, criterion='gini',
                       random_state=22, process=True, split=True):
    self.model = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes,
                                        criterion=criterion, random_state=random_state)
    if process:
      self.process()
    if split:
      self.split_datasets()
    self.model.fit(self.X_train, self.y_train)
    return self.model

  def get_score(self, method=accuracy_score, fit=False):
    if fit:
      self.get_fitted_model()
    preds = self.model.predict(self.X_test)
    return accuracy_score(self.y_test, preds)

  # X_test is either a stacked flattened array of shape 
  # (num_pixel, num_band_with_nd) or a stacked raster array of shape
  # (num_band_original, height, width)

  def predict(self, X_test, processing_needed=False):
    if processing_needed:
      sh = X_test.shape
      X_test = np.transpose(X_test.reshape(X_test.shape[0], X_test.shape[-1]*X_test.shape[-2]))
      X_test = add_nd(X_test)
      

    preds = self.model.predict(X_test)
    preds = pd.DataFrame(preds, columns=self.categories).idxmax(axis=1).to_numpy()

    if processing_needed:
      preds = preds.reshape(sh[1], sh[2])
    
    return preds
      


In [ ]:
ws = Workspace('stack.tif', esri_landcover_path, clip_area=wetland_bigger, resample_needed=True, trim_needed=5)

In [ ]:
ws.process(add_nd_needed=True)
start = time.time()
mm = ws.get_fitted_model(n_estimators=50, max_leaf_nodes=500, process=False)
# # mm = ws.get_fitted_model(n_estimators=10, process=False, split=False)
print(time.time() - start)
ws.get_score(fit=False, method=balanced_accuracy_score)

In [ ]:
full_prediction = ws.predict(ws.X)
# full_prediction = np.argmax(full_prediction, axis=1).reshape(357, 421)
full_prediction = full_prediction.reshape(357, 421)

In [ ]:
ep.plot_bands(full_prediction, cmap=cmap, norm=norm)

# Satellite Image segmentation with skimage

In [ ]:
from skimage import exposure
from skimage.segmentation import quickshift, slic, felzenszwalb
import PIL

In [ ]:
clipped_cover = clip_and_stack(esri_landcover_path, geojson=wetland_bigger, clip_only=True)[0]
clipped_ls8 = clip_and_stack('stack.tif', geojson=wetland_bigger, clip_only=True)[0]
clipped_cover_arr = get_arr_from_file(clipped_cover)
clipped_ls8_arr = get_arr_from_file(clipped_ls8)

In [ ]:
# show(clipped_ls8_arr, cmap=cmap, norm=norm)
plt.figure(figsize=(8,8))
show(clipped_cover_arr, cmap=cmap, norm=norm)

In [ ]:
cover_norm = exposure.rescale_intensity(np.dstack(clipped_ls8_arr))
# cover_norm_segments = slic(cover_norm, n_segments=5000, compactness=0.1)
cover_norm_segments = quickshift(cover_norm, max_dist=3, convert2lab=False)
# cover_norm_segments = felzenszwalb(cover_norm)
plt.figure(figsize=(8,8))
print(len(np.unique(cover_norm_segments)))
plt.imshow(cover_norm_segments, cmap='flag')